<a href="https://colab.research.google.com/github/kapeeshvarma/Book-Recommendation-System/blob/main/Book_Recommendation_System_Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**BOOK RECOMMENDATION SYSTEM**

Recommender systems are algorithms aimed at suggesting relevant items to users (movies, books, products). Recommender systems are really critical in some industries as they can generate a huge amount of income when they are efficient or also be a way to stand out significantly from competitors.

In this project, we create a book recommendation system for users through Unsupervised Machine Learning using three datasets - Books.csv, Users.csv, Ratings.csv

##Importing Data and Modules

In [1]:
# Import the necessary modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Read in the datasets
books = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Book Recommendation System - Kapeesh Varma/Books.csv')
users = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Book Recommendation System - Kapeesh Varma/Users.csv')
ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Book Recommendation System - Kapeesh Varma/Ratings.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
